In [7]:
from brian2 import *
import numpy as np

########################################
# 1. 網格與參數
Nx, Ny   = 20, 20
sigmaE   = 1.2
JE, JI   = 1.0, 0.2
tau      = 10*ms
alpha    = 0.4        # 角速度→平移增益
pitch_dot, roll_dot = 0.0, 0.0  # 後面可換成 time-varying TimedArray

########################################
# 2. 建 W 矩陣（扁平化成 N×N）
xs = np.arange(Nx)
ys = np.arange(Ny)
xx, yy = np.meshgrid(xs, ys, indexing='ij')
coords = np.vstack([xx.ravel(), yy.ravel()]).T
def torus_dist(a, b, L):
    d = np.abs(a-b)
    return np.minimum(d, L-d)

N = Nx*Ny
W = np.zeros((N, N))
for m,(xm,ym) in enumerate(coords):
    for n,(xn,yn) in enumerate(coords):
        dx = torus_dist(xm, xn, Nx)
        dy = torus_dist(ym, yn, Ny)
        W[m,n] = JE*np.exp(-(dx**2+dy**2)/(2*sigmaE**2)) - JI

########################################
# 3. Brian2 network
eqs = '''
dr/dt = (-r + clip(x,0,inf))/tau : 1
x = ge + ext : 1
dge/dt = -ge/tau : 1  # recurrent input
dext/dt = (-ext)/tau : 1  # velocity-driven bias
'''

G = NeuronGroup(N, eqs, method='euler')
G.r = 'exp(-((i//Ny-(Nx/2))**2 + (i%Ny-(Ny/2))**2)/4)'  # 初始中心 bump

# velocity bias（此處用 run_regularly，每 dt 更新一次）
@network_operation(dt=defaultclock.dt)
def move_bump():
    global pitch_dot, roll_dot
    # 例：0.5 秒後給一個 roll 角速度脈衝
    if 500*ms < defaultclock.t < 700*ms:
        roll_dot = 60/second
    else:
        roll_dot = 0/second
    # 近似偏導：右-左 / (2Δ)
    rmat = np.array(G.r).reshape((Nx,Ny))
    drdx = np.roll(rmat, -1, axis=0) - np.roll(rmat, 1, axis=0)
    drdy = np.roll(rmat, -1, axis=1) - np.roll(rmat, 1, axis=1)
    G.ext = alpha*(pitch_dot*drdx + roll_dot*drdy).ravel()

# 手動更新 recurrent connections，避免使用 spike 事件
@network_operation(dt=defaultclock.dt)
def update_synapses():
    # 手動更新 ge，模擬 recurrent synaptic input
    ge_input = np.zeros(N)
    for i in range(N):
        for j in range(N):
            ge_input[j] += W[j,i] * G.r[i]
    G.ge += ge_input * defaultclock.dt / tau

mon = StateMonitor(G, 'r', record=True, dt=20*ms)

run(1*second)


DimensionMismatchError: ext should be set with a dimensionless value, but got [ 0.  0. ... -0. -0.] Hz (unit is Hz).